In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler  # Add MinMaxScaler import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import joblib


In [2]:
# Load the dataset
df = pd.read_excel('DuplicateCleansing.xlsx')
# Preprocess text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Job Requirement'].fillna(''))

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['Job Requirement'].fillna(''))
sequence_lengths = [len(seq) for seq in sequences]

# Combine features
#maxlen = max(len(seq) for seq in sequences)
X_text = pad_sequences(sequences, padding='post', maxlen=2000)

label_encoder_education = LabelEncoder()
df['education'] = label_encoder_education.fit_transform(df['Pendidikan'])

scaler_experience = MinMaxScaler()
df['experience'] = scaler_experience.fit_transform(df[['Pengalaman Kerja']])

X_other = df[['experience', 'education']]
X = pd.concat([X_other, pd.DataFrame(X_text)], axis=1)

label_encoder_job_title = LabelEncoder()
y_encoded = label_encoder_job_title.fit_transform(df['Job Title'])

y_one_hot = to_categorical(y_encoded)



In [3]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Neural network model with Word Embedding
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=2002))  # Set input_length
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder_job_title.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Accuracy: {accuracy}")



Epoch 1/100


3/3 [==============================] - 4s 556ms/step - loss: 5.4293 - accuracy: 0.0127 - val_loss: 5.1148 - val_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 1s 251ms/step - loss: 4.8475 - accuracy: 0.0127 - val_loss: 4.8913 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 1s 243ms/step - loss: 4.6189 - accuracy: 0.0127 - val_loss: 4.7999 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 1s 254ms/step - loss: 4.5694 - accuracy: 0.0127 - val_loss: 4.8401 - val_accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 1s 249ms/step - loss: 4.5190 - accuracy: 0.0127 - val_loss: 4.7948 - val_accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 1s 257ms/step - loss: 4.4832 - accuracy: 0.0253 - val_loss: 4.8947 - val_accuracy: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 1s 250ms/step - loss: 4.4522 - accuracy: 0.0253 - val_loss: 4.9295 - val_acc

3/3 [==============================] - 1s 251ms/step - loss: 0.1882 - accuracy: 1.0000 - val_loss: 19.5760 - val_accuracy: 0.0000e+00
Epoch 53/100
3/3 [==============================] - 1s 237ms/step - loss: 0.1730 - accuracy: 1.0000 - val_loss: 19.7517 - val_accuracy: 0.0000e+00
Epoch 54/100
3/3 [==============================] - 1s 257ms/step - loss: 0.1473 - accuracy: 1.0000 - val_loss: 20.0961 - val_accuracy: 0.0000e+00
Epoch 55/100
3/3 [==============================] - 1s 248ms/step - loss: 0.1374 - accuracy: 1.0000 - val_loss: 20.5107 - val_accuracy: 0.0000e+00
Epoch 56/100
3/3 [==============================] - 1s 238ms/step - loss: 0.1228 - accuracy: 1.0000 - val_loss: 20.6978 - val_accuracy: 0.0000e+00
Epoch 57/100
3/3 [==============================] - 1s 246ms/step - loss: 0.1121 - accuracy: 1.0000 - val_loss: 20.7508 - val_accuracy: 0.0000e+00
Epoch 58/100
3/3 [==============================] - 1s 240ms/step - loss: 0.1064 - accuracy: 1.0000 - val_loss: 20.8439 - val_accur

In [4]:
# Save the trained model and preprocessing components
model.save("C:/Users/Asus/Document/nlp_model.h5")

# Save Tokenizer to JSON file
with open("C:/Users/Asus/Document/nlp_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer.to_json())

# Save Label Encoder for Job Title
label_encoder_job_title_JobReq = "C:/Users/Asus/Document/nlp_label_encoder_job_title.pkl"
joblib.dump(label_encoder_job_title, label_encoder_job_title_JobReq)

# Save Label Encoder for Education
label_encoder_education_JobReq = "C:/Users/Asus/Document/nlp_label_encoder_education.pkl"
joblib.dump(label_encoder_education, label_encoder_education_JobReq)

# Save MinMaxScaler for Experience
scaler_experience_filename = "C:/Users/Asus/Document/scaler_experience.pkl"
joblib.dump(scaler_experience, scaler_experience_filename)


C:\Users\Asus\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['C:/Users/Asus/Document/scaler_experience.pkl']